In [ ]:
import sklearn
assert sklearn.__version__ >= "0.20"

import numpy as np
import os

%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt

import dataretrieval

In [ ]:
# code for the lower chipichipi

areas